In [8]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from reproject import reproject_interp
from astropy import stats
from astropy.io import fits
from astropy.table import join
from glob import glob 
import sys
import os

sys.path.append('./../')
from modules import dendro_dendro, dendro_misc, dendro_props, dendro_mask

import warnings
warnings.filterwarnings('ignore')

from astropy.table import Table, QTable, join, vstack
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from reproject import reproject_interp

import astropy.units as au
from astropy import stats
from astrodendro import Dendrogram, pp_catalog
from astropy.wcs import WCS
from astropy.table import Column

from astropy.io import fits
import aplpy
from tqdm.auto import tqdm

from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve
from scipy.ndimage import binary_dilation, binary_closing

from astropy.wcs import WCS
from scipy import ndimage

from astropy.wcs import WCS
from astropy.io import fits
from tqdm.auto import tqdm
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, EllipseSkyRegion, Regions
import numpy as np

In [2]:
def get_pcperarcsec(sample_table):
    dist = sample_table['dist'].quantity[0]
    pcperarcsec = dist.to('pc').value/206265
    return(pcperarcsec*au.pc/au.arcsec)

def get_threshmask(data, rms, thresh=0):
    mask = data > (rms * thresh)
    return mask

def get_circmask(radius=5, h=11, w=11,):
    center = int(h/2)
    X, Y = np.ogrid[:h, :w]
    dist = np.sqrt((X - center)**2 + (Y-center)**2)
    mask = dist <= radius
    return mask *1

def get_prunemask(mask, thresh=10):

    mask_out = mask.copy()
    l, j = ndimage.label(mask_out)
    hist = ndimage.measurements.histogram(l, 0, j+1, j+1)
    os = ndimage.find_objects(l)

    for i in range(j):
        if hist[i+1]<thresh:
            mask_out[os[i]] = 0

    return(mask_out)

def get_regions(ra, dec, radius):

    center_sky = SkyCoord(ra, dec, unit='deg', frame='fk5')
    region_sky = [CircleSkyRegion(center, rad) for center, rad in zip(center_sky, radius)]
    region_sky = Regions(region_sky)

    return region_sky

def get_ds9regions(props_all, outputfile='ds9'):

    region_sky = get_regions(props_all['ra_com'], props_all['dec_com'], props_all['radius_circ'])
    region_sky.write(outputfile+'_com.reg', format='ds9', overwrite=True)

    region_sky = get_regions(props_all['ra_com'], props_all['dec_com'], np.ones(len(props_all['dec_com']))*(0.1/3600.)*au.deg)
    region_sky.write(outputfile+'_compoint.reg', format='ds9', overwrite=True)

    region_sky = get_regions(props_all['ra_max'], props_all['dec_max'], props_all['radius_circ'])
    region_sky.write(outputfile+'_max.reg', format='ds9', overwrite=True)

    region_sky = get_regions(props_all['ra_max'], props_all['dec_max'], np.ones(len(props_all['dec_com']))*(0.1/3600.)*au.deg)
    region_sky.write(outputfile+'_maxpoint.reg', format='ds9', overwrite=True)

    region_sky = get_regions(props_all['ra_dendro'], props_all['dec_dendro'], props_all['mean_sigma'])
    region_sky.write(outputfile+'_dendro.reg', format='ds9', overwrite=True)

    region_sky = get_regions(props_all['ra_dendro'], props_all['dec_dendro'], np.ones(len(props_all['mean_sigma']))*(0.1/3600.)*au.deg)
    region_sky.write(outputfile+'_dendropoint.reg', format='ds9', overwrite=True)

In [3]:
# Define names and filenames...

galaxy = 'ic5332'
galaxy_hst = galaxy
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha_sic.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name

cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/dendro' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)

rerun_all = False
rerun_masking = False

regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/Nebulae_catalogue_v3/Nebulae_catalogue_v3.fits' %root_dir

print(hstha_file)
print(muscat_file)
print(musha_file)
print(cutout_dir)
print(dendro_dir)
print(cutouts_hdus_dir)
print(regions_file)
print(regions_pickel_file)
print(sample_table_file)
print(muscat_table_file)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/hst_contsub/ic5332_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/muse/IC5332_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ic5332/muse/IC5332-0.87asec_MAPS.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/dendro
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts_hdus
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.reg
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.pickel
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/Nebulae_catalogue_v3/Nebulae_cata

In [4]:
# Loading files...
hstha_hdu = fits.open(hstha_file)[0]
muscat_hdu = fits.open(muscat_file)[0]

hstha_hdu = dendro_misc.convert_to_float32(hstha_hdu)
muscat_hdu = dendro_misc.convert_to_float32(muscat_hdu)

# Update arrays
muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
muscat_hdu.data[muscat_hdu.data==-1] = np.nan

# Interpolate masks
muscat_data_re, _ = reproject_interp(muscat_hdu, hstha_hdu.header)
muscat_data_mask = ~np.isnan(muscat_data_re)
data_outmask = hstha_hdu.data[~muscat_data_mask]

# Get RMS for whole map... 
std = stats.mad_std(data_outmask, ignore_nan=True)  # Get noise
std = stats.mad_std(data_outmask[data_outmask<20*std], ignore_nan=True)  # Get noise below threshold

In [12]:
mean = np.nanmean(data_outmask[data_outmask<20*std])  # Get noise below threshold
median = np.nanmedian(data_outmask[data_outmask<20*std])  # Get noise below threshold
std, mean, median

(91.6787315568802, 12.29103, 10.132849)

In [5]:
# Loading files...
hstha_hdu = fits.open(hstha_file)[0]
muscat_hdu = fits.open(muscat_file)[0]

hstha_hdu = dendro_misc.convert_to_float32(hstha_hdu)
muscat_hdu = dendro_misc.convert_to_float32(muscat_hdu)

# Update arrays
muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
muscat_hdu.data[muscat_hdu.data==-1] = np.nan

# Interpolate masks
muscat_data_re, _ = reproject_interp(muscat_hdu, hstha_hdu.header)
muscat_data_mask = ~np.isnan(muscat_data_re)
data_outmask = hstha_hdu.data[~muscat_data_mask]

# Get RMS for whole map... 
std = stats.mad_std(data_outmask, ignore_nan=True)  # Get noise
std = stats.mad_std(data_outmask[data_outmask<20*std], ignore_nan=True)  # Get noise below threshold

# Load regions, sample table and HDUs... 
hdus_cutouts = dendro_misc.load_pickle('%s/hdus_all.pickel' %cutout_dir)
regions = dendro_misc.load_pickle(regions_pickel_file)

sample_table = dendro_misc.get_galaxyprops(galaxy, sample_table_file)
muscat_table = dendro_misc.get_museprops(galaxy, muscat_table_file)

# Load cutout hdus with smoothed, masked, and non-masked data...
hdus_file = '%s/hdus_all_withmasked.pickel' %cutout_dir
muscat_regionIDs_file =  '%s/muscat_regionIDs.pickel' %cutout_dir

if os.path.exists(hdus_file) & ~rerun_masking:
    muscat_regionIDs = muscat_table['region_ID']
    hdus = dendro_misc.load_pickle(hdus_file)
else: 
    muscat_regionIDs = muscat_table['region_ID']
    hdus = dendro_dendro.get_maskedhdus(hdus_cutouts, regions, muscat_regionIDs)
    dendro_misc.save_pickle(hdus, hdus_file)

[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_all.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/sample.pickel
[INFO] [get_galaxyprops] Getting sample table properties for ic5332...
[INFO] [get_MuseProps] Getting MUSE catalouge properties for ic5332...
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ic5332/cutouts/hdus_all_withmasked.pickel


In [6]:
def get_hdumask(hdu_ha, hdu_mask, outputfile='tmp.fits'):
    
    # Set the shape of the reference H-alpha data and initialize its data array to -1
    shape_ha = hdu_ha.data.shape
    hdu_ha.data = np.ones(shape_ha) * -1
    wcs_ha = WCS(hdu_ha.header)

    # Loop over the HDUs provided in the 'indexmap_trunk_hdus_3sig' key
    for hdu_id in tqdm(range(len(hdu_mask)), desc='Masking regions'):

        # Count of non-negative values in the mask
        n_mask = np.sum(hdu_mask[hdu_id].data!=-1)  
        if n_mask == 0: 
            continue

        # Extract the current HDU mask and its WCS
        hdu_mask_ = hdu_mask[hdu_id]
        data_mask = (hdu_mask_.data != -1)  # Convert to a Boolean mask
        shape_mask = data_mask.shape
        wcs_mask = WCS(hdu_mask_.header)

        sky = wcs_mask.pixel_to_world(0,0)
        # Convert the sky coordinate to pixel coordinate in the reference H-alpha HDU
        pix = wcs_ha.world_to_pixel(sky)
        pix = np.array(pix, dtype=np.float64)
        x = round(pix[1])
        y = round(pix[0])
        w, h = hdu_mask_.data.shape
        hdu_ha.data[x:x+w, y:y+h][data_mask] = hdu_mask_.data[data_mask]

    # hdu_ha.data = np.int16(hdu_ha.data)
    hdu_ha.data = hdu_ha.data.astype('int16')

    # Write the resulting mask to an output file
    hdu_ha.writeto(outputfile, overwrite=True)

In [9]:
props_all = []
hdus_mask = []
hdus_mask_id = []
hdus_data_masked = []

for i in tqdm(range(len(muscat_regionIDs)), desc='Get sources:', position=0):
# for i in tqdm(range(50), desc='Get sources:', position=0):

    regionID = np.int16(muscat_regionIDs[i])
    data = hdus['hstha_hdu_smooth_masked'][i].data.copy()
    header = hdus['hstha_hdu_smooth_masked'][i].header.copy()

    mask_low = get_threshmask(data, std, thresh=1)
    mask_low_prune = get_prunemask(mask_low, thresh=100)
    mask_high = get_threshmask(data, std, thresh=4)
    mask_grow = binary_dilation(mask_high, iterations=-1, mask=mask_low_prune)
    mask_prune = get_prunemask(mask_grow, thresh=25)
    structure = get_circmask()
    mask_close = binary_closing(mask_prune, iterations=2, structure=structure)
    mask = mask_close.copy()

    hdu_mask = fits.PrimaryHDU(mask*1, header)
    hdus_mask += [hdu_mask]

    hdu_mask_id = hdu_mask.copy()
    hdu_mask_id.data = hdu_mask_id.data*i
    hdu_mask_id.data[~mask] = -1
    hdus_mask_id += [hdu_mask_id]

    data_masked = data.copy()
    data_masked[~mask] = np.nan
    hdu_data_masked = fits.PrimaryHDU(data_masked, header)
    hdus_data_masked += [hdu_data_masked]

    if np.nansum(~np.isnan(data_masked))==0:
        continue
    
    try:
        pixsize = np.array([np.abs(header['CDELT1']), np.abs(header['CDELT2'])]).mean() * au.degree
    except: 
        pixsize = np.array([np.abs(header['CD1_1']), np.abs(header['CD2_2'])]).mean() * au.degree

    if pixsize.value==1: 
        if 'CD1_1' in np.array(header.cards)[:,0]: 
            pixsize = np.array([np.abs(header['CD1_1']), np.abs(header['CD2_2'])]).mean() * au.degree
        elif 'PC1_1' in np.array(header.cards)[:,0]:
            pixsize = np.array([np.abs(header['PC1_1']), np.abs(header['PC2_2'])]).mean() * au.degree

    npix = np.nansum(mask) *au.pix
    flux = np.nansum(data_masked) *au.erg/au.s/au.cm**2
    flux_err = np.sqrt(npix)*std

    area_exact = npix.value*np.abs(pixsize.to(au.arcsec)**2)
    radius_circ = np.sqrt(area_exact/np.pi).to('arcsec')

    flux_max = np.nanmax(data_masked) *flux.unit
    flux_min = np.nanmin(data_masked) *flux.unit
    flux_mean = np.nanmean(data_masked) *flux.unit

    x_max, y_max = np.where(data_masked==flux_max.value.T)
    if len(x_max)>1 or len(y_max)>1:
        print('STOP') 
    else: 
        x_max, y_max = x_max[0], y_max[0]

    data_zeros = data_masked.copy()
    data_zeros[np.isnan(data_zeros)] = 0
    x_com, y_com = ndimage.center_of_mass(data_zeros)

    wcs = WCS(header)
    ra_max, dec_max = wcs.array_index_to_world_values([[y_max, x_max]])[0] *au.deg
    ra_com, dec_com = wcs.array_index_to_world_values([[y_com, x_com]])[0] *au.deg

    table_data = [
                regionID, x_max, y_max, x_com, y_com, ra_max, dec_max, ra_com, dec_com,
                npix, flux, flux_err, area_exact, radius_circ, flux_max, flux_min, flux_mean,
                ]
    table_data = [np.array(table_data_) for table_data_ in table_data]

    table_names = [
                'region_ID', 'x_max', 'y_max', 'x_com', 'y_com', 'ra_max', 'dec_max', 'ra_com', 'dec_com',
                'npix', 'flux', 'flux_err', 'area_exact', 'radius_circ', 'flux_max', 'flux_min', 'flux_mean',
                ]

    props = QTable(data=np.array(table_data), names=table_names)

    props['x_max'].unit = au.pix
    props['y_max'].unit = au.pix
    props['x_com'].unit = au.pix
    props['y_com'].unit = au.pix
    props['ra_max'].unit = ra_max.unit
    props['dec_max'].unit = dec_max.unit
    props['ra_com'].unit = ra_com.unit
    props['dec_com'].unit = ra_max.unit
    props['npix'].unit = au.pix
    props['flux'].unit = flux.unit
    props['flux_err'].unit = flux.unit
    props['area_exact'].unit = area_exact.unit
    props['radius_circ'].unit = radius_circ.unit
    props['flux_max'].unit = flux_max.unit
    props['flux_min'].unit = flux_min.unit
    props['flux_mean'].unit = flux_mean.unit

    pcperarcsec = get_pcperarcsec(sample_table)
    props['radius_circ_pc'] = props['radius_circ']*pcperarcsec

    props_all += [props]

    data_masked_zero = data_masked.copy()
    data_masked_zero[np.isnan(data_masked_zero)] = 0
    dendro = Dendrogram.compute(data_masked, min_delta=-1000, min_value=0, min_npix=0, wcs=wcs)

    metadata = {}
    metadata['data_unit'] = au.Jy / au.beam  # Dummy unit
    metadata['spatial_scale'] = pixsize.to('arcsec')
    metadata['beam_major'] = 0.1 * au.arcsec
    metadata['beam_minor'] = metadata['beam_major']

    props_dendro = pp_catalog(dendro.trunk, metadata, verbose=False)  
    props_dendro = QTable(props_dendro)
    props_dendro = props_dendro[np.nanargmax(props_dendro['flux'])]

    ra_dendro, dec_dendro = wcs.array_index_to_world_values([[props_dendro['x_cen'].value, props_dendro['y_cen'].value]])[0] *au.deg

    props['x_dendro'] = props_dendro['x_cen']
    props['y_dendro'] = props_dendro['y_cen']
    props['ra_dendro'] = ra_dendro
    props['dec_dendro'] = dec_dendro
    props['area_ellipse'] = props_dendro['area_ellipse']
    props['major_sigma'] = props_dendro['major_sigma']
    props['minor_sigma'] = props_dendro['minor_sigma']
    props['mean_sigma'] = props_dendro['radius']
    props['position_angle'] = props_dendro['position_angle']
    props['area_ellipse'] = props_dendro['area_ellipse']

props_all = vstack(props_all)
get_hdumask(hstha_hdu, hdus_mask_id, outputfile='tmp1.fits')

Get sources::   0%|          | 0/816 [00:00<?, ?it/s]

Masking regions:   0%|          | 0/816 [00:00<?, ?it/s]

In [10]:
get_ds9regions(props_all, outputfile='ds9')